# Point-by-point refined map visualisation notebook  
__Written by Haixing Fang, Jon Wright and James Ball__  
__Date: 21/02/2025__

This notebook will convert the (potentially) multi-valued results of a point-by-point strain refinement process to a single-valued 'TensorMap' with many useful export formats, like H5, Paraview XDMF, and MTEX CTF.

In [ ]:
import os

os.environ['OMP_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'

In [ ]:
exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())

In [ ]:
# this cell is tagged with 'parameters'
# to view the tag, select the cell, then find the settings gear icon (right or left sidebar) and look for Cell Tags

PYTHONPATH = setup_ImageD11_from_git( ) # ( os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

# dataset file to import
dset_path = 'si_cube_test/processed/Si_cube/Si_cube_S3DXRD_nt_moves_dty/Si_cube_S3DXRD_nt_moves_dty_dataset.h5'

# which phase to index
phase_str = 'Si'

# the minimum number of peaks you want a pixel to have to be counted
min_unique = 400

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize

from ImageD11.grain import grain
from ImageD11 import unitcell
import ImageD11.sinograms.dataset
from ImageD11.sinograms.point_by_point import PBPMap, PBPRefine
from ImageD11.sinograms.tensor_map import TensorMap
from ImageD11.nbGui import nb_utils as utils

%matplotlib ipympl

# Load data
## Dataset

In [ ]:
ds = ImageD11.sinograms.dataset.load(dset_path)
print(ds)

## Phases
If the parameter file was a json, we can access the unit cells via `ds.phases.unitcells`

In [ ]:
ds.phases = ds.get_phases_from_disk()
ds.phases.unitcells

In [ ]:
ref_ucell = ds.phases.unitcells[phase_str]
print(ref_ucell)

# Point-by-point refined map

In [ ]:
refmanpath = os.path.splitext(ds.refmanfile)[0] + f'_{phase_str}.h5'
refine = PBPRefine.from_h5(refmanpath)
refine.refinedmap.plot_nuniq_hist()

In [ ]:
# choose the minimum number of peaks you want a pixel to have to be counted
refine.refinedmap.choose_best(min_unique)
refine.refinedmap.plot_best(min_unique)

# Make TensorMap

In [ ]:
# now make a TensorMap from our refined map so we can plot and export
# let's make a phase id map from our pbpmap
phase_ids = TensorMap.recon_order_to_map_order(np.where(refine.refinedmap.best_nuniq > min_unique, 0, -1))
eps_sample = TensorMap.recon_order_to_map_order(refine.refinedmap.best_eps)
tmap = TensorMap.from_pbpmap(refine.refinedmap, steps=(1, ds.ystep, ds.ystep), phases={0: ref_ucell})
tmap['phase_ids'] = phase_ids
tmap['eps_sample'] = eps_sample
tmap.get_ipf_maps()
tmap.plot('nuniq')
tmap.plot('ipf_x')
tmap.plot('ipf_y')
tmap.plot('ipf_z')
eul = tmap.euler  # trigger calculation of all the maps

In [ ]:
# plot some (hopefully cleaner) strains

fig, axs = plt.subplots(3,3, sharex=True, sharey=True, layout='constrained', figsize=(10,10))
cmap = cm.get_cmap('RdBu_r')
normalizer = Normalize(-1e-3, 1e-3)
im = cm.ScalarMappable(norm=normalizer, cmap=cmap)
for i in range(3):
    for j in range(3):
        axs[i,j].imshow(tmap.eps_sample[0, ..., i, j], origin="lower", cmap=cmap, norm=normalizer, interpolation='nearest')
        axs[i,j].set_title(f'eps_{i+1}{j+1}')
fig.supxlabel('Lab X axis --->')
fig.supylabel('Lab Y axis --->')
fig.colorbar(im, ax=axs.ravel().tolist())
plt.show()

In [ ]:
# mean unitcell?
print(np.nanmean(tmap.unitcell[0, ...]))

In [ ]:
# if we have a previous tomographic TensorMap, we can try to get the labels map too:
try:
    tmap_tomo =  TensorMap.from_h5(ds.grainsfile, h5group='TensorMap_' + phase_str)
    tmap.add_map('labels', tmap_tomo.labels)
except (FileNotFoundError, OSError, KeyError):
    # couldn't find one, continue anyway
    pass

# Export
## TensorMap

In [ ]:
# save the refined TensorMap to disk

refined_tmap_path = os.path.join(ds.analysispath, f'{ds.sample}_{ds.dset}_refined_tmap_{phase_str}.h5')
tmap.to_h5(refined_tmap_path)
tmap.to_paraview(refined_tmap_path)

# MTEX

In [ ]:
# you can also do an MTEX export if you like:

refined_ctf_path = os.path.join(ds.analysispath, f'{ds.sample}_{ds.dset}_refined_tmap_{phase_str}.ctf')
tmap.to_ctf_mtex(refined_ctf_path, z_index=0)

In [ ]:
ds.save()